In [1]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib



In [2]:
# 4. Data Extraction
df = pd.read_csv(r'data_download.csv') # we import the data
print(df.columns) # showing columns

Index(['symbol', 'exchange', 'date', 'adjusted close', 'option symbol',
       'expiration', 'strike', 'call/put', 'style', 'ask', 'bid', 'mean price',
       'iv', 'volume', 'open interest', 'stock price for iv', '*', 'delta',
       'vega', 'gamma', 'theta', 'rho'],
      dtype='object')


In [3]:
# show unique values for each column
df.apply(pd.unique)

# We realize that the next columns are useless because they only have 1 value:
# 'symbol', 'exchange', '*', and 'style'

symbol                                                            [SPY]
exchange                                                     [NYSEArca]
date                  [07/30/2021, 07/29/2021, 07/28/2021, 07/27/202...
adjusted close        [438.51, 440.65, 438.83, 439.01, 441.02, 439.9...
option symbol         [SPY   210730C00215000, SPY   210730P00215000,...
expiration            [07/30/2021, 08/02/2021, 08/04/2021, 08/06/202...
strike                [215.0, 220.0, 225.0, 230.0, 235.0, 240.0, 245...
call/put                                                         [C, P]
style                                                               [A]
ask                   [224.56, 0.01, 219.56, 214.56, 209.56, 204.56,...
bid                   [223.35, 0.0, 218.35, 213.35, 208.35, 203.35, ...
mean price            [223.955, 0.0, 218.955, 213.955, 208.955, 203....
iv                    [-1.0, 0.361599, 0.384595, 0.351977, 0.328632,...
volume                [0, 50, 11, 2, 6, 13, 20, 14, 5, 4, 1, 3, 

In [4]:
# We proceed to drop the 4 unnecessary columns
df.drop(['symbol', 'exchange', '*','style'], axis=1, inplace=True)
df.columns

Index(['date', 'adjusted close', 'option symbol', 'expiration', 'strike',
       'call/put', 'ask', 'bid', 'mean price', 'iv', 'volume', 'open interest',
       'stock price for iv', 'delta', 'vega', 'gamma', 'theta', 'rho'],
      dtype='object')

In [5]:
# 'option symbol' is a column that we do not need because it is just a label. We proceed to drop the column
df.drop(['option symbol'], axis=1, inplace=True)
df. columns

Index(['date', 'adjusted close', 'expiration', 'strike', 'call/put', 'ask',
       'bid', 'mean price', 'iv', 'volume', 'open interest',
       'stock price for iv', 'delta', 'vega', 'gamma', 'theta', 'rho'],
      dtype='object')

In [6]:
# Find the number of numerical and categorical variables in our dataset
vars_num = [c for c in df.columns if df[c].dtypes!='O']

vars_cat = [c for c in df.columns if df[c].dtypes=='O']

print('Number of numerical features: {}'.format(len(vars_num)))
print('Number of categorical features: {}'.format(len(vars_cat)))

Number of numerical features: 14
Number of categorical features: 3


In [7]:
# Convert the 'date' column into datetime
df['date'] = pd.to_datetime(df['date'])
df['expiration'] = pd.to_datetime(df['expiration'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221046 entries, 0 to 221045
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                221046 non-null  datetime64[ns]
 1   adjusted close      221046 non-null  float64       
 2   expiration          221046 non-null  datetime64[ns]
 3   strike              221046 non-null  float64       
 4   call/put            221046 non-null  object        
 5   ask                 221046 non-null  float64       
 6   bid                 221046 non-null  float64       
 7   mean price          221046 non-null  float64       
 8   iv                  221046 non-null  float64       
 9   volume              221046 non-null  int64         
 10  open interest       221046 non-null  int64         
 11  stock price for iv  221046 non-null  float64       
 12  delta               221046 non-null  float64       
 13  vega                221046 no

In [8]:
# 5. Data Validation and Cleansing
df.isnull().sum() # We don't have null values

date                  0
adjusted close        0
expiration            0
strike                0
call/put              0
ask                   0
bid                   0
mean price            0
iv                    0
volume                0
open interest         0
stock price for iv    0
delta                 0
vega                  0
gamma                 0
theta                 0
rho                   0
dtype: int64

In [9]:
# 6. Data Aggregation and Representation
# We don't need to gather new features from other sources because we were provided with the dataset
# Create day, month, and year columns

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df['day_expiration'] = df['expiration'].dt.day
df['month_expiration'] = df['expiration'].dt.month
df['year_expiration'] = df['expiration'].dt.year

In [10]:
# Now we can drop 'date' column
df.drop(['date'], axis=1, inplace=True)
df.drop(['expiration'], axis=1, inplace=True)
df.columns

Index(['adjusted close', 'strike', 'call/put', 'ask', 'bid', 'mean price',
       'iv', 'volume', 'open interest', 'stock price for iv', 'delta', 'vega',
       'gamma', 'theta', 'rho', 'day', 'month', 'year', 'day_expiration',
       'month_expiration', 'year_expiration'],
      dtype='object')

In [11]:
df.apply(pd.unique)
# As the Month and Year only have 1 value, we can drop both columns

adjusted close        [438.51, 440.65, 438.83, 439.01, 441.02, 439.9...
strike                [215.0, 220.0, 225.0, 230.0, 235.0, 240.0, 245...
call/put                                                         [C, P]
ask                   [224.56, 0.01, 219.56, 214.56, 209.56, 204.56,...
bid                   [223.35, 0.0, 218.35, 213.35, 208.35, 203.35, ...
mean price            [223.955, 0.0, 218.955, 213.955, 208.955, 203....
iv                    [-1.0, 0.361599, 0.384595, 0.351977, 0.328632,...
volume                [0, 50, 11, 2, 6, 13, 20, 14, 5, 4, 1, 3, 151,...
open interest         [0, 1401, 1, 328, 1996, 3764, 1361, 769, 6085,...
stock price for iv    [438.97, 440.13, 438.62, 438.14, 441.05, 439.6...
delta                 [0.0, 1.0, 0.999999, -1e-06, 0.999995, -5e-06,...
vega                  [0.0, 1e-06, 9e-06, 4.9e-05, 6.8e-05, 9.2e-05,...
gamma                 [0.0, 2e-06, 9e-06, 1.2e-05, 1.6e-05, 2.2e-05,...
theta                 [0.0, -0.000546, -0.000569, -0.000592, -0.

In [12]:
# We drop Month and Year columns because they only have 1 unique value
df.drop(['month', 'year'], axis=1, inplace=True)
df.columns

Index(['adjusted close', 'strike', 'call/put', 'ask', 'bid', 'mean price',
       'iv', 'volume', 'open interest', 'stock price for iv', 'delta', 'vega',
       'gamma', 'theta', 'rho', 'day', 'day_expiration', 'month_expiration',
       'year_expiration'],
      dtype='object')

In [13]:
# As we only have 1 categorical variable (nominal)
# we need to apply one-hot-encoding to this variable ('call/put' column)
vars_num = [c for c in df.columns if df[c].dtypes!='O']

vars_cat = [c for c in df.columns if df[c].dtypes=='O']

print(vars_cat)
print(df[vars_cat[0]].unique())

['call/put']
['C' 'P']


In [15]:
# So we have to create 2 columns: 'C' and 'P' with 0 and 1 corresponding values
df.columns

Index(['adjusted close', 'strike', 'call/put', 'ask', 'bid', 'mean price',
       'iv', 'volume', 'open interest', 'stock price for iv', 'delta', 'vega',
       'gamma', 'theta', 'rho', 'day', 'day_expiration', 'month_expiration',
       'year_expiration'],
      dtype='object')

In [ ]:
# So we have to create 2 columns: 'C' and 'P' with 0 and 1 corresponding values
from sklearn.preprocessing import OneHotEncoder  

df2 = df.copy()


# Converting type of columns to category
df2['call/put'] = df2['call/put'].astype('category')


# Assigning numerical values and storing it in another columns
df2['c/p'] = df2['call/put'].cat.codes
  
# Create an instance of One-hot-encoder
enc = OneHotEncoder()
  
# Passing encoded columns
  
enc_data = pd.DataFrame(enc.fit_transform(df2[['c/p']]).toarray())

# Merge with main
df = df.join(enc_data)

df.rename(['0','1'], ['cp0','cp1'])
df.columns

In [ ]:
# 7. Data Analysis
df.shape

In [ ]:
df.describe()